In [20]:
%config IPCompleter.greedy=True

In [21]:
import os
import rasterio as rio
from datetime import datetime
import shutil
import fileinput
from rasterio.windows import Window
from matplotlib import pyplot
import subprocess

In [22]:
def get_date_from_filename(filename):
    date = datetime.strptime(filename[-15:-4], '%d-%b-%Y')
    return date

In [23]:
# constants coming from ee_export for later or new constants which will be included in the main running file later
drive_folder_location = r'C:\\Users\\karan\\Google Drive'
drive_folder = 'test27aug2'
satellite_choice = 'ls8'
block_size = 16 #automate later
resolution = 30

tif_paths = {
    'fine_res_path' : fr'{drive_folder_location}\\{drive_folder}_{satellite_choice}',
    'mask_path' : f'{drive_folder_location}\\{drive_folder}_{satellite_choice}_mask',
    'coarse_res_path' : f'{drive_folder_location}\\{drive_folder}_modis'
}
psrfm_info_dst = {
    'PSRFM_exe_path' : 'C:\\Users\\karan\\Documents\\CCRS_2A_COOP\\PSRFM_Wrapper\\PSRM_exe_params',
    'dst_base_path' : 'C:\\Users\\karan\\Documents\\CCRS_2A_COOP\\PSRFM_Testing\\AustraliaAutoTest2',
}
psrfm_info_dst['dst_input_path'] = psrfm_info_dst['dst_base_path'] + '\\input'
psrfm_info_dst['dst_param_path'] = psrfm_info_dst['dst_base_path'] + '\\params'
psrfm_info_dst['dst_output_path'] = psrfm_info_dst['dst_base_path'] + '\\output'
psrfm_info_dst['dst_temp_path'] = psrfm_info_dst['dst_base_path'] + '\\temp'



filenames = []
tile_struct = {
    'x_tiles' : 2,
    'y_tiles' : 2}

In [24]:
# Ensuring all local paths exist for PSRFM to run and for TIF files to be exported into
if not os.path.exists(psrfm_info_dst['dst_base_path']):
    os.mkdir(psrfm_info_dst['dst_base_path'])
if not os.path.exists(psrfm_info_dst['dst_input_path']):
    os.mkdir(psrfm_info_dst['dst_input_path'])
if not os.path.exists(psrfm_info_dst['dst_param_path']):
    os.mkdir(f'{psrfm_info_dst["dst_param_path"]}')
if not os.path.exists(f'{psrfm_info_dst["dst_base_path"]}\\PSRFM_Main.exe'):
    shutil.copy(f"{psrfm_info_dst['PSRFM_exe_path']}\\PSRFM_Main.exe", psrfm_info_dst['dst_param_path'])
if not os.path.exists(psrfm_info_dst['dst_output_path']):
    os.mkdir(f'{psrfm_info_dst["dst_output_path"]}')
if not os.path.exists(psrfm_info_dst['dst_temp_path']):
    os.mkdir(f'{psrfm_info_dst["dst_temp_path"]}')
# if not os.path.exists(f'{drive_folder_location}\\{drive_folder}'):
#     os.mkdir(f'{drive_folder_location}\\{drive_folder}')

In [25]:
# Cropping, converting to ENVI, and moving all the TIF files from GDrive to a local location with some small band removals or other processing to make files ready for PSRFM to be run on them
for path in tif_paths:
    for filename in os.listdir(tif_paths[path]):
        filenames.append(filename)
        with rio.open(f'{tif_paths[path]}\\{filename}') as image_to_crop:
            finalx = image_to_crop.meta["width"] - image_to_crop.meta["width"] % block_size
            finaly = image_to_crop.meta["height"] - image_to_crop.meta["height"] % block_size
            col_offset = (image_to_crop.width - finalx)/2
            row_offset = (image_to_crop.height - finaly)/2
            
            subset_window = Window(col_offset, row_offset, finalx, finaly)
            newargs = image_to_crop.meta.copy()
            newargs.update({
                'height': subset_window.height,
                'width': subset_window.width,
                'transform': rio.windows.transform(subset_window, image_to_crop.transform),
                'driver': 'ENVI'
            })
            if newargs['count'] == 7:
                newargs.update({'count' : 6})
            with rio.open(f'{psrfm_info_dst["dst_input_path"]}\\{filename[:-4]}.dat', 'w', **newargs) as dst:
                if 'ls8' in filename and 'mask' not in filename:
                    dst.write(image_to_crop.read(indexes=[1, 2, 3, 4, 5, 6], window=subset_window))
                else:
                    dst.write(image_to_crop.read(window=subset_window))

In [27]:
# finding the correct images to put into the param files and place all their metadata into the param files
for col in range(tile_struct['x_tiles']):
    for row in range(tile_struct['y_tiles']):
#         creating tuples of the filenames and dates corresponding to the current tile in {filename, date} format
        fine_res_filenames_dates = [(filename, get_date_from_filename(filename)) for filename in os.listdir(psrfm_info_dst['dst_input_path']) if filename.startswith(f'{satellite_choice}_{col}_{row}') and not filename.startswith(f'{satellite_choice}_mask_{col}_{row}') and filename.endswith('.dat')]
        mask_filenames_dates = [(filename, get_date_from_filename(filename)) for filename in os.listdir(psrfm_info_dst['dst_input_path']) if filename.startswith(f'{satellite_choice}_mask_{col}_{row}') and filename.endswith('.dat')]
        coarse_res_filenames_dates = [(filename, get_date_from_filename(filename)) for filename in os.listdir(psrfm_info_dst['dst_input_path']) if filename.startswith(f'modis_{col}_{row}') and filename.endswith('.dat')]
        
#         sorting the coarse and fine resolution files by date (second element in tuple) to help create "sets" of images for PSRFM processing
        coarse_res_filenames_dates = sorted(coarse_res_filenames_dates, key = lambda x: x[1])
        fine_res_filenames_dates = sorted(fine_res_filenames_dates, key = lambda x: x[1])
        mask_filenames_dates = sorted(mask_filenames_dates, key = lambda x: x[1])
#         fine_res_dates = sorted([name_date_tuple[1] for name_date_tuple in fine_res_filenames_dates])
#         
#         finding the indexes in coarse res images where both a fine res and coarse res image exist, which will then create pairs of start and end dates, and any coarse images inbetween will be used for predictions
        matching_date_indexes = []
        for pair in fine_res_filenames_dates:
            indx = [coarse_res_filenames_dates.index(tupl) for tupl in coarse_res_filenames_dates if tupl[1] == pair[1]]
            matching_date_indexes.append(indx[0])
#         creating "sets" for each start and end date pairing that's ready for PSRFM
        PSRFM_sets = []          
        for index in range(0, len(matching_date_indexes), 2):
#             determines all coarse res images for the current set
            coarse_res_set = coarse_res_filenames_dates[matching_date_indexes[index]:matching_date_indexes[index + 1] + 1]
#             the first coarse res image aligns with the the start fine res image, and the last coarse res image aligns with the end fine res image and same for the masks
            fine_res_set = [pair for pair in fine_res_filenames_dates if pair[1] == coarse_res_set[0][1] or pair[1] == coarse_res_set[-1][1]]
            mask_set = [pair for pair in mask_filenames_dates if pair[1] == coarse_res_set[0][1] or pair[1] == coarse_res_set[-1][1]]
#             appending all the filename, date pairs for this set of psrfm images
            PSRFM_sets.append({
                'coarse_res_filenames' : coarse_res_set,
                'fine_res_filenames': fine_res_set,
                'mask_filenames' : mask_set
            })
#     find how many times PSRFM has to be run and create param files for them
        PSRFM_instances = round(len(matching_date_indexes)/2)
        col_row_output_dir = f'{psrfm_info_dst["dst_output_path"]}\\tile_{col}_{row}'
        if not os.path.exists(col_row_output_dir):
            os.mkdir(col_row_output_dir)

        for instance in range(PSRFM_instances):
#             creating an output folder for each each instance
            ouptut_dir = f'{col_row_output_dir}\\instance_{instance+1}'
            if not os.path.exists(ouptut_dir):
                os.mkdir(ouptut_dir)
#             creating the parameter file for the current tile and instance of PSRFM
            if not os.path.exists(f'{psrfm_info_dst["dst_param_path"]}\\{col}_{row}_params_{instance + 1}.txt'):
                shutil.copy(f'{psrfm_info_dst["PSRFM_exe_path"]}\\psrfm_template.txt', f'{psrfm_info_dst["dst_param_path"]}\\{col}_{row}_params_{instance + 1}.txt')
#             now that file exists, insert the parameters

            with rio.open(f"{psrfm_info_dst['dst_input_path']}\\{PSRFM_sets[instance-1]['fine_res_filenames'][0][0]}") as img:
                nrows = img.meta['height']
                ncols = img.meta['width']
                nbands = img.meta['count']
            with fileinput.FileInput(f'{psrfm_info_dst["dst_param_path"]}\\{col}_{row}_params_{instance + 1}.txt', inplace = True) as paramfile:
                for line in paramfile:
                    if line.strip().startswith('IN_PAIR_COARSE_FNAME'):
                        print(f"IN_PAIR_COARSE_FNAME = {psrfm_info_dst['dst_input_path']}\\{PSRFM_sets[instance-1]['coarse_res_filenames'][0][0]} {psrfm_info_dst['dst_input_path']}\\{PSRFM_sets[instance-1]['coarse_res_filenames'][-1][0]}")
                    elif line.strip().startswith('IN_PAIR_FINE_FNAME'):
                        print(f"IN_PAIR_FINE_FNAME = {psrfm_info_dst['dst_input_path']}\\{PSRFM_sets[instance-1]['fine_res_filenames'][0][0]} {psrfm_info_dst['dst_input_path']}\\{PSRFM_sets[instance-1]['fine_res_filenames'][1][0]}")                                            
                    elif line.strip().startswith('IN_PAIR_FINE_MASK_FNAME'):
                        print(f"IN_PAIR_FINE_MASK_FNAME = {psrfm_info_dst['dst_input_path']}\\{PSRFM_sets[instance-1]['mask_filenames'][0][0]} {psrfm_info_dst['dst_input_path']}\\{PSRFM_sets[instance-1]['mask_filenames'][1][0]}")                                                     
                    elif line.strip().startswith('IN_PDAY_COARSE_NO'):
                        prediction_filenames = ''
                        for pair in PSRFM_sets[instance-1]['coarse_res_filenames'][1:-1]:
                            prediction_filenames += f" {psrfm_info_dst['dst_input_path']}\\{pair[0]}"
                        print(f"IN_PDAY_COARSE_NO = {len(PSRFM_sets[instance-1]['coarse_res_filenames']) - 2} \n {prediction_filenames}")
                    elif line.strip().startswith('OUT_PREDICTION_DIR'):
                        print(f"OUT_PREDICTION_DIR = {ouptut_dir}")
                    elif line.strip().startswith('OUT_TEMP_DIR'):
                        print(f"OUT_TEMP_DIR = {psrfm_info_dst['dst_temp_path']}")
                    elif line.strip().startswith('OUT_ENVI_HDR'):
                        print(f"OUT_ENVI_HDR = {psrfm_info_dst['dst_input_path']}\\{PSRFM_sets[instance-1]['fine_res_filenames'][0][0][:-4]}.hdr")
                    elif line.strip().startswith('NROWS'):
                        print(f"NROWS = {nrows}")
                    elif line.strip().startswith('COARSE_ROWS'):
                        print(f"COARSE_ROWS = {nrows}")
                    elif line.strip().startswith('NCOLS'):
                        print(f"NCOLS = {ncols}")
                    elif line.strip().startswith('COARSE_COLS'):
                        print(f"COARSE_COLS = {ncols}")
                    elif line.strip().startswith('NBANDS'):
                        print(f"NBANDS = {nbands}")
                    elif line.strip().startswith('RESOLUTION'):
                        print(f"RESOLUTION = {resolution}")
                    elif line.strip().startswith('BLOCK_SIZE'):
                        print(f"BLOCK_SIZE = {block_size}")
                    else:
                        print(line, end = '')
# fine_res_filenames_dates, mask_filenames_dates, coarse_res_filenames_dates, PSRFM_instances
# coarse_res_filenames_dates, matching_date_indexes, PSRFM_sets, len(matching_date_indexes)

In [28]:
for param_file in [filename for filename in os.listdir(psrfm_info_dst['dst_param_path']) if filename != 'PSRFM_Main.exe']:
    print(f'{psrfm_info_dst["dst_param_path"]}\\{param_file}')
    subprocess.run([f'{psrfm_info_dst["dst_param_path"]}\\PSRFM_Main.exe', f'{psrfm_info_dst["dst_param_path"]}\\{param_file}'])

C:\Users\karan\Documents\CCRS_2A_COOP\PSRFM_Testing\AustraliaAutoTest2\params\0_0_params_1.txt
C:\Users\karan\Documents\CCRS_2A_COOP\PSRFM_Testing\AustraliaAutoTest2\params\0_1_params_1.txt
C:\Users\karan\Documents\CCRS_2A_COOP\PSRFM_Testing\AustraliaAutoTest2\params\1_0_params_1.txt
C:\Users\karan\Documents\CCRS_2A_COOP\PSRFM_Testing\AustraliaAutoTest2\params\1_1_params_1.txt
